Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic Regression

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_l2 = tf.placeholder(tf.float32)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros(num_labels))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + beta_l2 * tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:offset + batch_size,:]
        batch_label = train_labels[offset:offset + batch_size,:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_label, beta_l2:1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step%500 == 0:
            print("Minibatch Loss at step %d: %f" % (step, l))
            print("Minibatch Accuracy: %.1f%%" % accuracy(predictions, batch_label))
            print("Validation Accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test set Accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch Loss at step 0: 21.004559
Minibatch Accuracy: 8.6%
Validation Accuracy: 9.4%
Minibatch Loss at step 500: 2.632106
Minibatch Accuracy: 75.8%
Validation Accuracy: 75.9%
Minibatch Loss at step 1000: 1.697345
Minibatch Accuracy: 77.3%
Validation Accuracy: 78.5%
Minibatch Loss at step 1500: 1.304674
Minibatch Accuracy: 77.3%
Validation Accuracy: 79.5%
Minibatch Loss at step 2000: 0.910530
Minibatch Accuracy: 82.0%
Validation Accuracy: 80.6%
Minibatch Loss at step 2500: 0.645776
Minibatch Accuracy: 85.2%
Validation Accuracy: 81.5%
Minibatch Loss at step 3000: 0.836647
Minibatch Accuracy: 82.8%
Validation Accuracy: 81.9%
Test set Accuracy: 89.3%


### 1 Layer NN

In [7]:
batch_size = 128
hidden_layers = 1024

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_l2 = tf.placeholder(tf.float32)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_layers]))
    biases1 = tf.Variable(tf.zeros(hidden_layers))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_layers, num_labels]))
    biases2 = tf.Variable(tf.zeros(num_labels))
    
    hidden_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    logits = tf.matmul(hidden_train, weights2) + biases2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + beta_l2 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_valid, weights2) + biases2)
    hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_test, weights2) + biases2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:offset + batch_size,:]
        batch_label = train_labels[offset:offset + batch_size,:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_label, beta_l2:1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step%500 == 0:
            print("Minibatch Loss at step %d: %f" % (step, l))
            print("Minibatch Accuracy: %.1f%%" % accuracy(predictions, batch_label))
            print("Validation Accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test set Accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch Loss at step 0: 696.910400
Minibatch Accuracy: 15.6%
Validation Accuracy: 38.7%
Minibatch Loss at step 500: 193.818024
Minibatch Accuracy: 78.9%
Validation Accuracy: 81.0%
Minibatch Loss at step 1000: 115.312531
Minibatch Accuracy: 82.8%
Validation Accuracy: 81.8%
Minibatch Loss at step 1500: 68.376289
Minibatch Accuracy: 81.2%
Validation Accuracy: 82.8%
Minibatch Loss at step 2000: 41.354225
Minibatch Accuracy: 87.5%
Validation Accuracy: 85.1%
Minibatch Loss at step 2500: 25.058931
Minibatch Accuracy: 91.4%
Validation Accuracy: 86.1%
Minibatch Loss at step 3000: 15.448546
Minibatch Accuracy: 85.2%
Validation Accuracy: 86.7%
Test set Accuracy: 93.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
batch_size = 128
hidden_layers = 1024

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_l2 = tf.placeholder(tf.float32)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_layers]))
    biases1 = tf.Variable(tf.zeros(hidden_layers))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_layers, num_labels]))
    biases2 = tf.Variable(tf.zeros(num_labels))
    
    hidden_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    logits = tf.matmul(hidden_train, weights2) + biases2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + beta_l2 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_valid, weights2) + biases2)
    hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_test, weights2) + biases2)

In [11]:
num_steps = 51
num_batches = 3

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        offset = step % num_batches
        batch_data = train_dataset[offset:offset + batch_size,:]
        batch_label = train_labels[offset:offset + batch_size,:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_label, beta_l2:1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step%8 == 0:
            print("Minibatch Loss at step %d: %f" % (step, l))
            print("Minibatch Accuracy: %.1f%%" % accuracy(predictions, batch_label))
            print("Validation Accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test set Accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch Loss at step 0: 642.215454
Minibatch Accuracy: 5.5%
Validation Accuracy: 32.2%
Minibatch Loss at step 8: 313.164062
Minibatch Accuracy: 98.4%
Validation Accuracy: 66.8%
Minibatch Loss at step 16: 309.761719
Minibatch Accuracy: 100.0%
Validation Accuracy: 66.8%
Minibatch Loss at step 24: 307.292969
Minibatch Accuracy: 100.0%
Validation Accuracy: 66.8%
Minibatch Loss at step 32: 304.843750
Minibatch Accuracy: 100.0%
Validation Accuracy: 66.8%
Minibatch Loss at step 40: 302.414185
Minibatch Accuracy: 100.0%
Validation Accuracy: 66.8%
Minibatch Loss at step 48: 300.003906
Minibatch Accuracy: 100.0%
Validation Accuracy: 66.8%
Test set Accuracy: 73.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
batch_size = 128
hidden_layers = 1024

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_l2 = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_layers]))
    biases1 = tf.Variable(tf.zeros(hidden_layers))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_layers, num_labels]))
    biases2 = tf.Variable(tf.zeros(num_labels))
    
    hidden_train_drop = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), keep_prob=keep_prob)
    logits = tf.matmul(hidden_train_drop, weights2) + biases2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + beta_l2 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    hidden_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    train_prediction = tf.nn.softmax(tf.matmul(hidden_train, weights2) + biases2)
    hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_valid, weights2) + biases2)
    hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_test, weights2) + biases2)

In [15]:
num_steps = 51
num_batches = 3

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        offset = step % num_batches
        batch_data = train_dataset[offset:offset + batch_size,:]
        batch_label = train_labels[offset:offset + batch_size,:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_label, beta_l2:1e-3, keep_prob:0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step%8 == 0:
            print("Minibatch Loss at step %d: %f" % (step, l))
            print("Minibatch Accuracy: %.1f%%" % accuracy(predictions, batch_label))
            print("Validation Accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test set Accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch Loss at step 0: 831.439697
Minibatch Accuracy: 9.4%
Validation Accuracy: 32.9%
Minibatch Loss at step 8: 315.147736
Minibatch Accuracy: 99.2%
Validation Accuracy: 70.2%
Minibatch Loss at step 16: 310.407257
Minibatch Accuracy: 100.0%
Validation Accuracy: 70.0%
Minibatch Loss at step 24: 307.943634
Minibatch Accuracy: 100.0%
Validation Accuracy: 70.4%
Minibatch Loss at step 32: 305.499390
Minibatch Accuracy: 100.0%
Validation Accuracy: 71.6%
Minibatch Loss at step 40: 304.765137
Minibatch Accuracy: 100.0%
Validation Accuracy: 71.1%
Minibatch Loss at step 48: 300.665253
Minibatch Accuracy: 100.0%
Validation Accuracy: 71.0%
Test set Accuracy: 78.5%


So here our new model with dropout is capable of better generalization than previous overfitting model by 5%

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [33]:
batch_size = 128
num_hidden1 = 1024
num_hidden2 = 256

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_l2 = tf.placeholder(tf.float32)
    keep_prob1 = tf.placeholder(tf.float32)
    keep_prob2 = tf.placeholder(tf.float32)
    global_step = tf.Variable(0) 
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden1], stddev=np.sqrt(2.0 / (image_size * image_size))))
    biases1 = tf.Variable(tf.zeros(num_hidden1))
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden1, num_hidden2], stddev=np.sqrt(2.0 / num_hidden1)))
    biases2 = tf.Variable(tf.zeros(num_hidden2))
    
    weights3 = tf.Variable(tf.truncated_normal([num_hidden2, num_labels], stddev=np.sqrt(2.0 / num_hidden2)))
    biases3 = tf.Variable(tf.zeros(num_labels))
    
    hidden1_train_drop = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    hidden2_train_drop = tf.nn.relu(tf.matmul(hidden1_train_drop, weights2) + biases2)
    logits = tf.matmul(hidden2_train_drop, weights3) + biases3
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + \
    beta_l2 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
    
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    hidden1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    hidden2_train = tf.nn.relu(tf.matmul(hidden1_train, weights2) + biases2)
    train_prediction = tf.nn.softmax(tf.matmul(hidden2_train, weights3) + biases3)
    hidden1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    hidden2_valid = tf.nn.relu(tf.matmul(hidden1_valid, weights2) + biases2)
    valid_prediction = tf.nn.softmax(tf.matmul(hidden2_valid, weights3) + biases3)
    hidden1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    hidden2_test = tf.nn.relu(tf.matmul(hidden1_test, weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(hidden2_test, weights3) + biases3)
    

In [34]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:offset + batch_size,:]
        batch_label = train_labels[offset:offset + batch_size,:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_label, beta_l2:1e-3, keep_prob1:0.5, keep_prob2:0.8}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step%500 == 0:
            print("Minibatch Loss at step %d: %f" % (step, l))
            print("Minibatch Accuracy: %.1f%%" % accuracy(predictions, batch_label))
            print("Validation Accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test set Accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch Loss at step 0: 3.380093
Minibatch Accuracy: 6.2%
Validation Accuracy: 43.7%
Minibatch Loss at step 500: 1.118487
Minibatch Accuracy: 83.6%
Validation Accuracy: 86.3%
Minibatch Loss at step 1000: 0.823058
Minibatch Accuracy: 88.3%
Validation Accuracy: 87.5%
Minibatch Loss at step 1500: 0.759332
Minibatch Accuracy: 85.2%
Validation Accuracy: 88.0%
Minibatch Loss at step 2000: 0.557507
Minibatch Accuracy: 90.6%
Validation Accuracy: 88.5%
Minibatch Loss at step 2500: 0.470500
Minibatch Accuracy: 93.0%
Validation Accuracy: 88.8%
Minibatch Loss at step 3000: 0.560995
Minibatch Accuracy: 88.3%
Validation Accuracy: 89.0%
Test set Accuracy: 94.6%
